In [2]:
%config InteractiveShell.ast_node_interactivity = 'all'

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc
import pickle
import wandb

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchinfo import summary

from layer.kan_layer import KANLinear, NewGELU
from sklearn.preprocessing import StandardScaler
from utils.data_utils import StormDataset, prepare_data

from utils.data_utils import prepare_data
from utils.model_utils import train_val, test
from utils.load_dataset import DataModule
from model.rnn_kan_v1_1 import RNN_KAN
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

In [ ]:
# in_features = (20, 50, 100)
# hidden_features = 100
# output_features = 1
# seq_len = 5
# n_ahead = 4
# batch_size = 128

# model = RNN_KAN(in_features, hidden_features, output_features, n_ahead)

# summary(model, (batch_size, seq_len, in_features[0]))

In [ ]:
data_path = 'data/cma_era5'
raw_train_data, raw_val_data, raw_test_data = load_and_process_data(data_path=data_path, train_years=list(range(1980, 2017)), val_years=[2017, 2018, 2019], test_years=[2020, 2021, 2022])

In [ ]:
#====================== Train model ======================#
# config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {torch.cuda.get_device_name(0)} for device")

# hyperparams
in_features = (20, 50, 100)
hidden_features = 100
output_features = 1
seq_len = 5
n_aheads = list(range(1, 13))
batch_size = 128
epochs = 10
num_workers = 0
pin_memory = True

results = {}

for n_ahead in n_aheads:
  print(f"================== TRAINING n_ahead = {n_ahead} ==================")
  X_train, y_train, metadata_train = prepare_data(raw_train_data, sequence_length = 5, n_ahead = n_ahead)
  X_val, y_val, metadata_val = prepare_data(raw_val_data, sequence_length = 5, n_ahead = n_ahead)
  X_test, y_test, metadata_test = prepare_data(raw_test_data, sequence_length = 5, n_ahead = n_ahead)

  print(f"X_shape: {X_train.shape}")
  print(f"y_shape: {y_train.shape}")

  raw_test_data_2020 = {k: v for k, v in raw_test_data.items() if int(k[-4:]) == 2020}
  raw_test_data_2021 = {k: v for k, v in raw_test_data.items() if int(k[-4:]) == 2021}
  raw_test_data_2022 = {k: v for k, v in raw_test_data.items() if int(k[-4:]) == 2022}

  X_test_2020, y_test_2020, metadata_test_2020 = prepare_data(raw_test_data_2020, sequence_length = 5, n_ahead = n_ahead)
  X_test_2021, y_test_2021, metadata_test_2021 = prepare_data(raw_test_data_2021, sequence_length = 5, n_ahead = n_ahead)
  X_test_2022, y_test_2022, metadata_test_2022 = prepare_data(raw_test_data_2022, sequence_length = 5, n_ahead = n_ahead)

  scaler_X = StandardScaler()
  X_train_scaled = scaler_X.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
  X_val_scaled   = scaler_X.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
  X_test_scaled  = scaler_X.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
  X_test_2020_scaled = scaler_X.transform(X_test_2020.reshape(-1, X_test_2020.shape[-1])).reshape(X_test_2020.shape)
  X_test_2021_scaled = scaler_X.transform(X_test_2021.reshape(-1, X_test_2021.shape[-1])).reshape(X_test_2021.shape)
  X_test_2022_scaled = scaler_X.transform(X_test_2022.reshape(-1, X_test_2022.shape[-1])).reshape(X_test_2022.shape)

  scaler_y = StandardScaler()
  y_train_scaled = scaler_y.fit_transform(y_train)
  y_val_scaled   = scaler_y.transform(y_val)
  y_test_scaled  = scaler_y.transform(y_test)
  y_test_2020_scaled = scaler_y.transform(y_test_2020)
  y_test_2021_scaled = scaler_y.transform(y_test_2021)
  y_test_2022_scaled = scaler_y.transform(y_test_2022)

  train_dataset = StormDataset(X_train_scaled, y_train_scaled)
  val_dataset = StormDataset(X_val_scaled, y_val_scaled)
  test_dataset = StormDataset(X_test_scaled, y_test_scaled)
  test_dataset_2020 = StormDataset(X_test_2020_scaled, y_test_2020_scaled)
  test_dataset_2021 = StormDataset(X_test_2021_scaled, y_test_2021_scaled)
  test_dataset_2022 = StormDataset(X_test_2022_scaled, y_test_2022_scaled)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
  test_2020_loader = DataLoader(test_dataset_2020, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
  test_2021_loader = DataLoader(test_dataset_2021, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
  test_2022_loader = DataLoader(test_dataset_2022, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

  model = RNN_KAN(in_features, hidden_features, output_features, n_ahead)
  model = model.to(device)
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=1e-3)
  model, total_time, train_loss_per_epoch, val_loss_per_epoch = train_val(model, criterion, optimizer, train_loader, val_loader, device, batch_size, epochs)

  test_mae = test(model, test_loader, scaler_y, device)
  test_2020_mae = test(model, test_2020_loader, scaler_y, device)
  test_2021_mae = test(model, test_2021_loader, scaler_y, device)
  test_2022_mae = test(model, test_2022_loader, scaler_y, device)

  results[n_ahead] = {
      "mae": [test_mae, test_2020_mae, test_2021_mae, test_2022_mae],
      "time": [total_time]
  }

Using NVIDIA GeForce RTX 4060 Ti for device
================== TRAINING n_ahead = 1 ==================
X_shape: (25864, 5, 20)
y_shape: (25864, 1)
batch_x shape: torch.Size([128, 5, 20])
batch_y shape: torch.Size([128, 1])
Epoch 1/10, Train Loss: 0.1389, Val Loss: 0.0359
Epoch 2/10, Train Loss: 0.0413, Val Loss: 0.0363
Epoch 3/10, Train Loss: 0.0385, Val Loss: 0.0325
Epoch 4/10, Train Loss: 0.0374, Val Loss: 0.0307
Epoch 5/10, Train Loss: 0.0366, Val Loss: 0.0303
Epoch 6/10, Train Loss: 0.0359, Val Loss: 0.0313
Epoch 7/10, Train Loss: 0.0361, Val Loss: 0.0312
Epoch 8/10, Train Loss: 0.0353, Val Loss: 0.0288
Epoch 9/10, Train Loss: 0.0351, Val Loss: 0.0336
Epoch 10/10, Train Loss: 0.0349, Val Loss: 0.0311
================== TRAINING n_ahead = 2 ==================
X_shape: (24788, 5, 20)
y_shape: (24788, 2)
batch_x shape: torch.Size([128, 5, 20])
batch_y shape: torch.Size([128, 2])
Epoch 1/10, Train Loss: 0.2012, Val Loss: 0.0834
Epoch 2/10, Train Loss: 0.0827, Val Loss: 0.0693
Epoch 3/1

In [31]:
for k,v in results.items():
  print(f"Prediction {k*6}h: ")
  print(f"\t Total time: {v['time'][0]:.2f}s")
  print(f"\t Test MAE: {v['mae'][0]:.4f}")
  print(f"\t Test 2020 MAE: {v['mae'][1]:.4f}")
  print(f"\t Test 2021 MAE: {v['mae'][2]:.4f}")
  print(f"\t Test 2022 MAE: {v['mae'][3]:.4f}")

Prediction 6h: 
	 Total time: 43.06s
	 Test MAE: 2.2912
	 Test 2020 MAE: 1.6587
	 Test 2021 MAE: 1.5704
	 Test 2022 MAE: 3.9090
Prediction 12h: 
	 Total time: 55.58s
	 Test MAE: 2.8469
	 Test 2020 MAE: 2.0908
	 Test 2021 MAE: 2.0694
	 Test 2022 MAE: 4.6908
Prediction 18h: 
	 Total time: 53.81s
	 Test MAE: 3.5166
	 Test 2020 MAE: 2.5090
	 Test 2021 MAE: 2.5497
	 Test 2022 MAE: 5.9112
Prediction 24h: 
	 Total time: 66.94s
	 Test MAE: 4.2953
	 Test 2020 MAE: 3.0632
	 Test 2021 MAE: 2.8761
	 Test 2022 MAE: 7.5970
Prediction 30h: 
	 Total time: 134.02s
	 Test MAE: 5.5395
	 Test 2020 MAE: 3.6863
	 Test 2021 MAE: 3.8055
	 Test 2022 MAE: 10.0045
Prediction 36h: 
	 Total time: 140.82s
	 Test MAE: 5.6256
	 Test 2020 MAE: 4.0229
	 Test 2021 MAE: 3.7308
	 Test 2022 MAE: 10.1154
Prediction 42h: 
	 Total time: 146.26s
	 Test MAE: 5.9525
	 Test 2020 MAE: 4.3287
	 Test 2021 MAE: 4.1533
	 Test 2022 MAE: 10.4040
Prediction 48h: 
	 Total time: 150.58s
	 Test MAE: 5.9761
	 Test 2020 MAE: 4.7251
	 Test 202

## Short-term forecasting

In [3]:
file_paths = [
    # 'artifacts/raw_ETTh1-v0/raw_ETTh1.csv',
    # 'artifacts/raw_ETTh2-v0/raw_ETTh2.csv',
    # 'artifacts/raw_ETTm1-v0/raw_ETTm1.csv',
    'artifacts/raw_electricity-v0/raw_electricity.csv',
    'artifacts/raw_exchange_rate-v0/raw_exchange_rate.csv',
    # 'artifacts/raw_PEMS03-v0/raw_PEMS03.pkl',
    # 'artifacts/raw_PEMS04-v0/raw_PEMS04.pkl',
    # 'artifacts/raw_PEMS07-v0/raw_PEMS07.pkl',
    # 'artifacts/raw_PEMS08-v0/raw_PEMS08.pkl'
]
#====================== Train model ======================#
# config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {torch.cuda.get_device_name(0)} for device")

# hyperparams
in_features = (20, 50, 100)
hidden_features = 100

window_size = 168
n_aheads = [3, 6, 12, 24]
epochs = 10
num_workers = 0
pin_memory = True

results = {}
for file_path in file_paths:
  if file_path == 'artifacts/raw_electricity-v0/raw_electricity.csv':
    batch_size = 32
  if file_path == 'artifacts/raw_electricity-v0/raw_exchange_rate.csv':
    batch_size = 4
  for n_ahead in n_aheads:
    print(f"================== TRAINING n_ahead = {n_ahead} ==================")
    dm = DataModule(
      file_path=file_path,
    )

    dm.setup()
    dm.scale()
    train_loader, val_loader, test_loader = dm.get_dataloader(batch_size=batch_size, window_size=window_size, horizon=n_ahead)
    sample_x, sample_y = dm.train_dataset[0]

    in_features = (sample_x.shape[-1], 50, 100)
    out_features = sample_y.shape[-1]

    model = RNN_KAN(in_features, hidden_features, out_features, n_ahead)
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    model, total_time, train_loss_per_epoch, val_loss_per_epoch = train_val(model, criterion, optimizer, train_loader, val_loader, device, batch_size, epochs)

    metrics = [nn.L1Loss()]
    mae, mse = test(model, test_loader, dm, device)

    results[n_ahead] = {
        "RSE": mae,
        "CORR": mse,
        "time": [total_time]
    }

Using NVIDIA GeForce RTX 4060 Ti for device
================== TRAINING n_ahead = 3 ==================
raw_electricity
batch_x shape: torch.Size([32, 168, 321])
batch_y shape: torch.Size([32, 3, 321])
Epoch 1/10, Train Loss: 0.3164, Val Loss: 1.5371
Epoch 2/10, Train Loss: 0.1788, Val Loss: 1.4956
Epoch 3/10, Train Loss: 0.1516, Val Loss: 1.5054
Epoch 4/10, Train Loss: 0.1374, Val Loss: 1.5164
Epoch 5/10, Train Loss: 0.1289, Val Loss: 1.5003
Epoch 6/10, Train Loss: 0.1225, Val Loss: 1.5313
Epoch 7/10, Train Loss: 0.1173, Val Loss: 1.5154
Epoch 8/10, Train Loss: 0.1131, Val Loss: 1.5133
Epoch 9/10, Train Loss: 0.1100, Val Loss: 1.5054
Epoch 10/10, Train Loss: 0.1075, Val Loss: 1.5337
================== TRAINING n_ahead = 6 ==================
raw_electricity
batch_x shape: torch.Size([32, 168, 321])
batch_y shape: torch.Size([32, 6, 321])
Epoch 1/10, Train Loss: 0.3843, Val Loss: 1.7139
Epoch 2/10, Train Loss: 0.2336, Val Loss: 1.7624
Epoch 3/10, Train Loss: 0.1937, Val Loss: 1.7620
Epoc

In [4]:
results

{3: {'RSE': tensor(0.2462),
  'CORR': tensor(-0.1243),
  'time': [1218.8609449863434]},
 6: {'RSE': tensor(0.3214), 'CORR': tensor(nan), 'time': [1266.7999322414398]},
 12: {'RSE': tensor(0.3364),
  'CORR': tensor(-0.0477),
  'time': [1145.5743157863617]},
 24: {'RSE': tensor(0.3430),
  'CORR': tensor(-0.0155),
  'time': [1950.2187852859497]}}